## Demo of geopandas:
### Calculate total quarter-mile service area of all heavy-rail Red Line stations

In [ ]:
import geopandas as gp

In [ ]:
def make_gdf_from_wfs_request(base_url, typename_parm):
    request_url = base_url + '?' + 'service=wfs'
    request_url += '&request=getfeature'
    request_url += '&typename=' + typename_parm
    request_url += '&outputformat=json'
    gdf = gp.read_file(request_url)
    return gdf

In [ ]:
# MassGIS WFS base URL
massgis_base_url = 'https://gis-prod.digital.mass.gov/geoserver/wfs'

In [ ]:
stations_gdf = make_gdf_from_wfs_request(massgis_base_url, 'massgis:GISDATA.MBTA_NODE')

In [ ]:
stations_gdf.plot()

In [ ]:
stations_gdf.explore()

In [ ]:
stations_gdf.head()

In [ ]:
# - Query for  Red line stations (remember, some stations carry more than one line!)
stations_red = stations_gdf[stations_gdf['line'].str.contains('RED')]

In [ ]:
stations_red.plot()

In [ ]:
# Filter out Mattapan Trolley stations
stations_red = stations_red[~stations_red['route'].str.contains('Mattapan')]

In [ ]:
stations_red.plot()

In [ ]:
# Calculate the quarter-mile service area of each staion
quarter_mile_in_meters = 0.25 *  1609.344
buf = stations_red.buffer(quarter_mile_in_meters)

In [ ]:
buf.head()

In [ ]:
# 'buf' is a  GeoSeries, turn it into a GeoDataFrame
buf_gdf = gp.GeoDataFrame(gp.GeoSeries(buf))
buf_gdf = buf_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')

In [ ]:
buf_gdf.head()

In [ ]:
buf_gdf.explore()

In [ ]:
# Dissolve buffers
buf_gdf = buf_gdf.dissolve()

In [ ]:
buf_gdf.explore()

In [ ]:
# Get total of all quarter-mile service areas, in square miles
# Recall: SRS of data uses 'meters' as unit of measure
sq_meters_to_sq_miles_factor = 0.00000038610
buf_gdf['area_sq_mi'] = buf_gdf.area * sq_meters_to_sq_miles_factor

In [ ]:
buf_gdf